In [2]:
%load_ext autoreload
%autoreload 2

import os
import pathlib
import operator
import math
import json
import re
import datetime
import typing
import functools
import collections
import pickle

import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate

import onnx
import onnxruntime
import torch
import torchvision

from torchvision.models import resnet50, mobilenetv3
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

from painting_estimation.images.utils import image_size, ImgSize
from painting_estimation.images.preprocessing import ImagePreprocessor
from painting_estimation.inference.inference import ONNXModel

In [5]:
MODEL_PATH = pathlib.Path.cwd().parent / "models"
DATA_PATH = pathlib.Path.cwd().parent / "data"

In [4]:
EFF_NET_B3 = MODEL_PATH / "eff_net_b3.onnx"

In [7]:
train_path = DATA_PATH / "from_me" / "artsynet"

train_anno = pd.read_csv(train_path / "artsynet.csv", index_col=0)
train_data = sorted((train_path / "images").glob("*.jpg"), key=lambda x: int(x.stem.rsplit("-", 1)[1]))

In [11]:
EFF_NET_PREPROCESSOR = ImagePreprocessor(
    target_size=ImgSize(width=300, height=300),
    target_dim_order=(2, 0 ,1),
    target_dtype=np.float32,
    interpolation=cv2.INTER_LINEAR,
    to_bgr=False,
    extra_batch_dim=0,
    normalize=True,
    means=(0.485, 0.456, 0.406),
    stds=(0.229, 0.224, 0.225),
    initial_size_before_crop=ImgSize(width=320, height=320),
)

In [12]:
EFF_NET_EXTRACTOR = ONNXModel(EFF_NET_B3) 

In [13]:
AXIS = 0, 1
QUANTILES = [0.1, 0.25, 0.5, 0.75, 0.9]

def regular_features(img):
    img_size = image_size(img)
    
    features = []
    features.extend(img.mean(AXIS).tolist())
    features.extend(img.std(AXIS).tolist())
    features.extend(np.quantile(img, q=QUANTILES, axis=AXIS).ravel().tolist())
    
    return features

In [19]:
def add_target(train_anno: pd.DataFrame) -> pd.DataFrame:
    range_mask = train_anno["sale_message"].str.find("–").ne(-1)

    train_anno["price1"] = "min"
    train_anno["price2"] = "max"
    train_anno["p1"] = 0.0
    train_anno["p2"] = 0.0

    train_anno["price_range"] = train_anno["sale_message"].str.split("–")

    train_anno.loc[range_mask, "price1"] = train_anno.loc[range_mask, "price_range"].transform(operator.itemgetter(0))
    train_anno.loc[range_mask, "price2"] = train_anno.loc[range_mask, "price_range"].transform(operator.itemgetter(1))

    train_anno.loc[~range_mask, "price2"] = train_anno.loc[~range_mask, "price_range"].transform(
        operator.itemgetter(0)
    )

    invalid_mask = train_anno["price2"].astype(bool)

    for ccy, coef in zip(["US$", "€", "£"], [1.0, 1.1, 1.3]):
        ccy_mask = train_anno["price2"].str.startswith(ccy)
        invalid_mask &= ~ccy_mask
        train_anno.loc[ccy_mask, "p2"] = \
        train_anno.loc[ccy_mask, "price2"].str.strip(ccy).str.replace(",", "").replace(".", "").astype(float)
        ccy_range_mask = ccy_mask & range_mask
        train_anno.loc[ccy_range_mask, "p1"] = \
        train_anno.loc[ccy_range_mask, "price1"].str.strip(ccy).str.replace(",", "").replace(".", "").astype(float)

        train_anno.loc[ccy_range_mask, "p2"] += train_anno.loc[ccy_range_mask, "p1"]
        train_anno.loc[ccy_range_mask, "p2"] /= 2

        train_anno.loc[ccy_mask, "p2"] *= coef
        
    train_anno["target"] = train_anno["p2"].values
    train_anno["invalid"] = invalid_mask
    
    return train_anno

In [20]:
def extract_features_target(train_anno, train_path, preprocessor, feature_extractor):
    
    train_anno = add_target(train_anno)
    
    X = []
    X_add = []
    y = []
    visited = set()
    
    for p, t, not_valid in tqdm(
        zip(train_anno["local_image"], train_anno["target"], train_anno["invalid"]), total=len(train_anno)
    ):
        
        if not_valid or p in visited:
            continue
            
        path = train_path / "images" / p
        img = cv2.imread(str(path))
        if img is None:
            continue
        
        # RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        features = feature_extractor(preprocessor(img))
        X.append(features.ravel())
        X_add.append(regular_features(img))
        y.append(t)
        
        visited.add(p)
        
    X = np.concatenate([np.asarray(X), np.asarray(X_add)], axis=1)
    y = np.asarray(y)
    
    return X, y

In [21]:
X, y = extract_features_target(
    train_anno, 
    train_path, 
    preprocessor=EFF_NET_PREPROCESSOR, 
    feature_extractor=EFF_NET_EXTRACTOR,
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20580/20580 [38:22<00:00,  8.94it/s]


In [22]:
x_path = DATA_PATH / "X_eff2.pkl"
with open(str(x_path), "wb") as f:
    pickle.dump(X, f)

In [23]:
y_path = DATA_PATH / "y_eff2.pkl"
with open(str(y_path), "wb") as f:
    pickle.dump(y, f)